## Setup

In [1]:
%load_ext autoreload
%autoreload 2
import sys 
import os 
import pandas as pd
sys.path.append('../scripts')

from data_cleaner import extract_features, impute_missing_values

In [2]:
processed_path = os.path.join('..', 'data', 'processed', 'df_merged.csv')
df = pd.read_csv(processed_path)

C:\Users\vscerra\AppData\Local\Temp\ipykernel_16200\3432328469.py:2: DtypeWarning: Columns (19,21,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(processed_path)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10820820 entries, 0 to 10820819
Data columns (total 29 columns):
 #   Column        Dtype  
---  ------        -----  
 0   HH1           float64
 1   HAZ           float64
 2   UB2           float64
 3   EC6           float64
 4   EC7           float64
 5   EC8           float64
 6   EC9           float64
 7   EC10          float64
 8   EC11          float64
 9   EC12          float64
 10  EC13          float64
 11  EC14          float64
 12  EC15          float64
 13  BD2           object 
 14  BD3           object 
 15  CA1           object 
 16  WB6A          object 
 17  WB6B          object 
 18  WAGE          object 
 19  MN5           object 
 20  WB14          object 
 21  MN34          object 
 22  WS1           object 
 23  WS4           object 
 24  HH48          float64
 25  windex5       object 
 26  stunted       int64  
 27  ecd_i_count   float64
 28  ecd_on_track  int64  
dtypes: float64(15), int64(2), object(12)
memory

In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
HH1,10820820.0,1591.220124,974.971713,1.00,695.00,1517.00,2476.00,3220.00
HAZ,10115820.0,-0.739214,2.477369,-9.98,-1.87,-1.09,-0.25,9.99
UB2,10116500.0,2.020472,1.412093,0.00,1.00,2.00,3.00,4.00
EC6,4120700.0,0.291145,0.454290,0.00,0.00,0.00,1.00,1.00
EC7,4127980.0,0.310074,0.462524,0.00,0.00,0.00,1.00,1.00
EC8,4127900.0,0.297648,0.457224,0.00,0.00,0.00,1.00,1.00
EC9,4119560.0,0.920700,0.270206,0.00,1.00,1.00,1.00,1.00
EC10,4130940.0,0.233816,0.423257,0.00,0.00,0.00,0.00,1.00
EC11,4131360.0,0.861382,0.345547,0.00,1.00,1.00,1.00,1.00
EC12,4132920.0,0.851635,0.355461,0.00,1.00,1.00,1.00,1.00


In [5]:
cat_cols = ['BD2', 'BD3', 'CA1', 'WB6A', 'WB6B', 'WAGE', 'MN5', 'WB14', 'MN34', 'windex5', 'WS1', 'WS4']

for col in cat_cols:
    print(f"\n--- {col} ---")
    print(df[col].value_counts(dropna=False))


--- BD2 ---
BD2
YES    5721900
NaN    4837620
NO      260080
DK        1220
Name: count, dtype: int64

--- BD3 ---
BD3
NaN    5098920
YES    4734700
NO      986840
DK         360
Name: count, dtype: int64

--- CA1 ---
CA1
NO             9418700
NaN             704320
YES             691820
DK                5680
NO RESPONSE        300
Name: count, dtype: int64

--- WB6A ---
WB6A
SECONDARY/ HIGHER SECONDARY    3333280
PRIMARY                        2349500
NaN                            2335260
LOWER SECONDARY                2093340
HIGHER                          687040
ECE                              22400
Name: count, dtype: int64

--- WB6B ---
WB6B
NaN                             2357660
SSC/Dakhil                      1675560
GRADE 5                         1415100
GRADE 8                         1041100
GRADE 9                          751780
HSC/Alim/Diploma/Polytechnic     670380
GRADE 7                          613140
GRADE 6                          439100
GRADE 3           

## Summarize Missingness

In [6]:
missing_summary = df.isnull().sum().sort_values(ascending=False)
missing_pct = (missing_summary / len(df)) * 100

missing_df = pd.DataFrame({
    'Missing Count': missing_summary,
    'Missing %': missing_pct
})

missing_df[missing_df['Missing Count'] > 0]

,Missing Count,Missing %
MN34,10017220,92.573576
MN5,9472220,87.536989
WS4,8605091,79.523465
WB14,6813180,62.963620
EC9,6701260,61.929318
EC6,6700120,61.918782
EC15,6696420,61.884589
EC8,6692920,61.852244
EC7,6692840,61.851505
EC10,6689880,61.824150


In [7]:
df_model = extract_features(df)

# drop rows missing more than 50% of selected columns
threshold = int(0.3 * len(df_model.columns))
df_cleaned = df_model.dropna(thresh=(len(df_model.columns) - threshold))

print(f"Original rows: {len(df_model)}")
print(f"Rows after dropping high-missing: {len(df_cleaned)}")

Original rows: 10820820
Rows after dropping high-missing: 3763450


In [8]:
drop_features = ['MN34', 'MN5', 'WS4']
df_cleaned.drop(drop_features, axis=1, inplace=True)
df_cleaned.head()

C:\Users\vscerra\AppData\Local\Temp\ipykernel_16200\1873568886.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.drop(drop_features, axis=1, inplace=True)


,UB2,EC6,EC7,EC8,EC9,EC10,EC11,EC12,EC13,EC14,...,WAGE,WB14,windex5,HH48,WS1,BD2,BD3,CA1,stunted,ecd_on_track
0,4.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,...,20-24,NaN,Richest,4.0,TUBE WELL / BOREHOLE,NaN,NaN,NO,0,1
1,4.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,...,20-24,NaN,Middle,3.0,TUBE WELL / BOREHOLE,NaN,NaN,NO,0,1
2,4.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,...,20-24,NaN,Middle,3.0,TUBE WELL / BOREHOLE,NaN,NaN,NO,0,1
3,4.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,...,20-24,NaN,Middle,2.0,TUBE WELL / BOREHOLE,NaN,NaN,NO,0,1
4,4.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,...,20-24,NaN,Fourth,3.0,TUBE WELL / BOREHOLE,NaN,NaN,NO,0,1


In [9]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3763450 entries, 0 to 10819819
Data columns (total 23 columns):
 #   Column        Dtype  
---  ------        -----  
 0   UB2           float64
 1   EC6           float64
 2   EC7           float64
 3   EC8           float64
 4   EC9           float64
 5   EC10          float64
 6   EC11          float64
 7   EC12          float64
 8   EC13          float64
 9   EC14          float64
 10  EC15          float64
 11  WB6A          object 
 12  WB6B          object 
 13  WAGE          object 
 14  WB14          object 
 15  windex5       object 
 16  HH48          float64
 17  WS1           object 
 18  BD2           object 
 19  BD3           object 
 20  CA1           object 
 21  stunted       int64  
 22  ecd_on_track  int64  
dtypes: float64(12), int64(2), object(9)
memory usage: 689.1+ MB


## Impute Remaining Missing Values

Use median for numerical columns and mode for categorical columns

In [10]:
df_imputed = impute_missing_values(df_cleaned.copy())

In [11]:
save_path = os.path.join('..', 'data', 'processed')
df_imputed.to_csv(os.path.join(save_path, "df_imputed.csv"), index = False)
df_imputed.head(10)

,UB2,EC6,EC7,EC8,EC9,EC10,EC11,EC12,EC13,EC14,...,WAGE,WB14,windex5,HH48,WS1,BD2,BD3,CA1,stunted,ecd_on_track
0,4.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,...,20-24,CANNOT READ AT ALL,Richest,4.0,TUBE WELL / BOREHOLE,NaN,NaN,NO,0,1
1,4.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,...,20-24,CANNOT READ AT ALL,Middle,3.0,TUBE WELL / BOREHOLE,NaN,NaN,NO,0,1
2,4.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,...,20-24,CANNOT READ AT ALL,Middle,3.0,TUBE WELL / BOREHOLE,NaN,NaN,NO,0,1
3,4.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,...,20-24,CANNOT READ AT ALL,Middle,2.0,TUBE WELL / BOREHOLE,NaN,NaN,NO,0,1
4,4.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,...,20-24,CANNOT READ AT ALL,Fourth,3.0,TUBE WELL / BOREHOLE,NaN,NaN,NO,0,1
5,4.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,...,20-24,CANNOT READ AT ALL,Middle,4.0,TUBE WELL / BOREHOLE,NaN,NaN,NO,0,1
6,4.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,...,20-24,CANNOT READ AT ALL,Richest,5.0,TUBE WELL / BOREHOLE,NaN,NaN,NO,0,1
7,4.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,...,20-24,CANNOT READ AT ALL,Richest,4.0,TUBE WELL / BOREHOLE,NaN,NaN,NO,0,1
8,4.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,...,20-24,CANNOT READ AT ALL,Richest,3.0,TUBE WELL / BOREHOLE,NaN,NaN,NO,0,1
9,4.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,...,20-24,CANNOT READ AT ALL,Richest,3.0,TUBE WELL / BOREHOLE,NaN,NaN,NO,0,1
